# Introduction

During the 2021 United Nations Climate Change Conference a report was published stating that 'Air pollution is the largest environmental risk to the public’s health, contributing to cardiovascular disease, lung cancer and respiratory diseases. It is costing the UK economy £20 billion a year and contributes to over 25,000 deaths a year' $^{1}$. However, this report could be masking a bigger issue of air polution worldwide. An OECD report published in 2016 states that air pollution costs 1% of GDP each year and accounts for 9 million premature deaths $^{2}$.

With the human population growing at an exponential rate, it would seem, we now find ourselves in a continous loop. The Malthusian model, commonly used in development econmics, looks to model population growth. As a country grows food production, basic industry and infrastructure grow to service the population until an equilibrium level is reached. There are two takeaways from the model relating to air pollution. Firstly, air pollution does not cost and is in essence free to 'produce' implying there is no constraint on air pollution. Moreover, as less economically developed countries accelerate towards their equilibrium state, industry will have to keep up and air pollution is predicted to get worse.

This being said, there appears to be other factors that affect air pollution. Global warming is well documented, however, the secondary and tertiary impacts on air pollution are less clear. Wildfires raged across Greece in August 2021, with the problem stemming from unusually hot and long summers. The Guardian reported that 'the fires were some of the worst on record' $^{3}$. Wildfires produce high amounts of black carbon, carbon dioxide, ozone precursors and black carbon into the atmosphere, causing air quality to plummet. 

However, with evolving winds, humidity levels and precipitation rates, areas once crippled by air pollution may soon see clearer skies while air pollution is involuntarily dumped onto other cities in a game of air pollution roulette. 

In this report we will look in depth at the factors above namely: demographics, weather and events. The report will aim to highlight areas of interest related to these factors in three countries: Mexico, UK and Iceland - representing an emerging nation and two developed nations with very different weather dynamics, industry and natural resources.  

The report will focus upon 2 key measurements particulate matter 10 (pm10) and particulate matter 25 (pm25) these are defined as particulate matter that is smaller than 10 micrometres and 25 micrometres respecitivley. These were chosen given the breadth of activities that produce pm10 and pm25. The main contributor to pm10 and pm25 is the burning of oil, fuel and wood, however, pm25 and pm10 also include dust from industrial activity, landfill sites and even smaller bacteria. The wide range of polluting activities emitting these particulates will allow us to examine, explore and analyse the three nations in depth. 

Finally, we aim to predict air quality in a certain city for a certain day given the weather conditions using simple regression model.



$^{1}$ http://www.local.gov.uk/parliament/briefings-and-responses/cop26-and-impact-air-pollution-public-health-and-wellbeing-house

$^{2}$ OECD (2016), The Economic Consequences of Outdoor Air Pollution, OECD Publishing, Paris, https://doi.org/10.1787/9789264257474-en.

$^{3}$ https://www.theguardian.com/world/2021/sep/30/its-like-a-war-greece-battles-increase-in-summer-wildfires

# Data & Measurements

Most of the data in this report and prediction model comes from two apis. One for weather and one for pollution.
We wrapped both of them in functions that call the apis, parse the response and return appropriate data frames.
In addition to returning the data asked for, our functions also store the data in CSV files under ./data in the project directory. This is done to reduce network requests and also to be able to verify and play around with the data in raw format when exploring it. 
<br>If a csv file exists for the city the user asks for, and it contains all the rows for the queried dates, they are returned and the api is not called.

We also have yearly population data. That is stored locally in csv files, no api there(something more plz).

## Weather
The source for weather data is the meteostat api found here https://dev.meteostat.net/python/. It has a convenient python library that allows the user to query for historical weather data for a particular location during a specified time interval.
The python library even automatically collects data from different weather stations and bundles them together. 
For the purpose of this report, we need to transform the data a bit before using it. For example, we are only interested in a subset of all the data in the response. Furthermore, to ease usage of our model, our users should only provide a city name but the meteostat library only accepts latitude and longitude coordinates. 
To facilitate that, prior to using the meteostat api, we call a geocoding api from open-meteo.com [https://open-meteo.com/en/docs/geocoding-api]. It accepts a city name and returns all cities that match the name and their geographic latitude and longitude coordinates. 

The results from the open-meteo api are global. This means that if multiple cities share the same name, they are all in the API response. Because of this, we filter the results down to a particular country, and assume that if our user ask for 'London' she means London UK, not London New York USA. In addition, of all the cities that match a given country, we assume the user wants the most populated city.

Armed with the latitude and longitude for the city the user wants to know about, we then finally call the meteostat api. 
As mentioned before, if we already have CSV data locally for the range, instead of going through all that was mentioned above,
we just return appropriate data from the CSV file.

## Pollution

Pollution data is taken from 


## Demography

In [43]:
import sys
from pandas.core.frame import DataFrame
import requests
import pandas as pd
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import openaq
import warnings
import numpy as np


from datetime import datetime as dt
from datetime import date
from datetime import timedelta
from meteostat import Point
from meteostat import Daily 
# Focus on these 5 cities and document their contry code
cities_of_interest = {"Akureyri": "IS", "London": "GB", 
    "Mexico City": "MX", "Newcastle": "GB", "Reykjavík":"IS"}
data_dirs = {"weather": "data/weather", "pollution": "data/pollution"}

def fetch_weather_data(city_name, date_from=date.today() - timedelta(30), date_to=date.today()):
    ''' Fetch weather measurements for a particular city during a particular time.

    '''
    #We limit to the 5 cities above
    city_country = get_city_country(city_name)

    #Since both the weather api and pandas dataframe operate on the datetime level
    #cast date to datetime early 
    dt_from = cast_date_to_datetime(date_from)
    dt_to = cast_date_to_datetime(date_to)

    weather = fetch_data_from_csv('weather', city_country[0], dt_from= dt_from, dt_to=dt_to)

    if(weather.empty):
        weather = fetch_data_from_api('weather', city_country, dt_from, dt_to)
    #The complicated case is: a file exists but not all the data is there
    elif(needs_more_data(weather, dt_from, dt_to)):
        weather = partial_csv_update('weather', weather, city_country)
    return weather

def fetch_pollution_data(city_name, date_from=date.today() - timedelta(days=30), date_to=date.today()):

    city_country = get_city_country(city_name)

    #Since both the weather api and pandas dataframe operate on the datetime level
    #cast date to datetime early 
    dt_from = cast_date_to_datetime(date_from)
    dt_to = cast_date_to_datetime(date_to)

    pollution = fetch_data_from_csv('pollution', city_country[0], dt_from = dt_from, dt_to = dt_to)

    if(pollution.empty):
        pollution = fetch_data_from_api('pollution', city_country, dt_from, dt_to)
    elif(needs_more_data(pollution, dt_from, dt_to)):
        pollution = partial_csv_update('pollution', pollution, city_country)
    return pollution 

def cast_date_to_datetime(date):
    midnight_time = dt.min.time()
    return dt.combine(date, midnight_time)

def needs_more_data(data, dt_from, dt_to):
    delta_days = (dt_to - dt_from).days + 1
    data_subset = data[(dt_from <= data.index) & (data.index <= dt_to)]
    return len(data_subset) < delta_days

def get_city_country(city_name):
    if city_name not in cities_of_interest.keys():
        raise Exception(f"Please enter one of {cities_of_interest}.")
    return (city_name, cities_of_interest[city_name])

def fetch_data_from_csv(type, city_name, **kwargs):
    file_dir = data_dirs[type]
    
    #Empty dataframe for when there is no csv file
    data = pd.DataFrame({'' : []})

    try:
        data = pd.read_csv(f'{file_dir}/{city_name}.csv', index_col='date', parse_dates=['date'])
        if (kwargs):
            return data[(kwargs['dt_from'] <= data.index) & (data.index <= kwargs['dt_to'])]
    except FileNotFoundError:
        print(f"No historical {type} data found locally. Using API to get fresh data.\n")
    return data

def fetch_data_from_api(type, city_and_country, date_from, date_to):
    # The first part of the tuple is the city and our csv data is organized by city name.
    city_coords = query_lat_long(city_and_country)

    if(type=='weather'):
        data = query_historical_weather(city_coords, date_from, date_to)
    elif(type=='pollution'):
        data = query_historical_pollution(city_and_country, date_from, date_to)

    #Write new csv file
    dir_name = data_dirs[type]
    data.to_csv(f'{dir_name}/{city_and_country[0]}.csv')
    return data

def partial_csv_update(type, from_api, city_and_country):
    #Note that this is all of our CSV data for the city. no time filter
    from_file = fetch_data_from_csv(type, city_and_country[0])

    total_data = pd.concat([from_file, from_api])
    unique_days = total_data.drop_duplicates()
    unique_days = unique_days.sort_values('date')

    dir_name = data_dirs[type]
    unique_days.to_csv(f'{dir_name}/{city_and_country[0]}.csv')
    return unique_days

def filter_results_to_country(geoapi_response_data, country):
    #If there are multiple cities with the same name, we choose the most populated
    filter_by_country = [resp for resp in geoapi_response_data if resp['country_code'] == country]
    # 
    sorted_by_pop = sorted(filter_by_country, key = lambda resp: resp['population'] if 'population' in resp else 0, reverse=True)
    result = sorted_by_pop[0]

    # Relevant subset of the result dict
    return {key: result[key] for key in ('latitude', 'longitude', 'elevation', 'population')}

def query_lat_long(city_country):
    params_dict = {
        'name': city_country[0],
        #Default is 10 but since newcastle gives 9, we might hit the limit
        'count': 100
    }

    # Since the user gives us more than 3 chars, the api performs fuzzy matching. So we do not
    # need to worry abt spelling
    resp = requests.get('https://geocoding-api.open-meteo.com/v1/search', params_dict)
    data = resp.json()
    
    return filter_results_to_country(data['results'], city_country[1])

def query_historical_weather(lat_long_elevation, date_from, date_to):
    # Since we have dates and the api uses time, we need to convert from date to datetime
    midnight_time = dt.min.time()
    dt_from = dt.combine(date_from, midnight_time)
    dt_to = dt.combine(date_to, midnight_time)
    
    location = Point(
        lat_long_elevation['latitude'], 
        lat_long_elevation['longitude'], 
        lat_long_elevation['elevation'])

    daily = Daily(location, start=dt_from, end=dt_to)
    
    # Ask meteostat to fill in any gaps in the data
    daily.normalize()
    data = daily.fetch()
    
    #tavg=Temp average (C).prcp=Total precipitation(mm). wdir=Wind direction(degrees)
    #wspd=Average wind speed(km/h).wpgt=Wind peak gust(km/hr). pres=Sea-level air pressure(hpa)
    #rhum=Relative humidity(does not work)
    response = data[['tavg', 'prcp', 'wdir', 'wspd', 'wpgt', 'pres']]

    #Since time is an index, simply calling rename does not work
    tidy = response.rename_axis(index={"time": "date"})
    #tidy['date'] = pd.to_datetime(tidy['date'], format='%y%m%d')

    return tidy

def get_weather(city_country):
    coords = query_lat_long(city_country)
    return query_historical_weather(coords)


#From fetching measurements data notebook

# TODO
# 2. The dates are not the same for all cities.. Look into that
# 3. Maybe count locations and provide that into the dataframe as well. 
#    It's relevent to know how many measures are in the city.
# 4. Can we choose certain type of measures

# Filter what city we want to get
def filter_results_to_country(geoapi_response_data, country):
    #If there are multiple cities with the same name, we choose the most populated
    filter_by_country = [resp for resp in geoapi_response_data if resp['country_code'] == country]
    # 
    sorted_by_pop = sorted(filter_by_country, key = lambda resp: resp['population'] if 'population' in resp else 0, reverse=True)
    result = sorted_by_pop[0]

    # Relevant subset of the result dict
    return {key: result[key] for key in ('latitude', 'longitude', 'elevation', 'population')}

def query_historical_pollution(city_country, date_from, date_to):
    '''
    This function takes in a city, parameter and date and writes data into a csv.file
    Input:
        city: name of a city (string)
        ???parameter: List of strings that represent the parameters wanted to calculate
        date_from: measurments after this date will be calculated
        date_to: Measures until this date will be calculated
    '''
    
    # Explicitly use v2 of the api http://dhhagan.github.io/py-openaq/api.html
    api = openaq.OpenAQ(version ='v2')
    # Get the longitude and latitude for the city
    location = query_lat_long(city_country)
    coords = f'{location["latitude"]},{location["longitude"]}'
    
    # Call the location api to check for the first date updated
    locations = api.locations(coordinates = coords,radius = 10000,df = True)

    min_date = locations["firstUpdated"].min()
    min_date = min_date.tz_convert(None)
    min_date = pd.to_datetime(min_date) 
    
    if min_date > date_from:
        date_from = min_date  
    
    # Number of days we want measurements for
    day_diff = (date_to - date_from).days
    
    # How we split the call between days to the API
    split_days = 30

    # Number of 30 day blocks in our range
    number_months = day_diff // split_days

    # Initialize the start date
    start = date_from

    # Add measurements data frame 30 days at a time
    # An extra iteration for the remaining <30 days
    for n in range(number_months + 1):
        
        # Find the end date
        end = start + timedelta(days = split_days)
        
        # Fetch the data from the measurment api
        try:
            df_api = api.measurements(coordinates = coords, radius = 10000, df = True, 
                                        limit = 30000, parameter = ["pm25", "pm10"], value_from = 0,
                                    date_from = start, date_to = end)
        except KeyError:
            print(f"Opanaq API request with coordinates {coords} and date range {start}-{end} raised exception.\n That can mean there is no data in that range.")
            return pd.DataFrame({'' : []})

        # Start as the last end date
        start = end

        # For the first iteration create df
        if n == 0: 
            df = df_api.copy()
        # After the first iteration append the data
        else:
            df = df.append(df_api)
    
    ## Data prepping 

    # Change the index
    df.index.name = 'Date.local'
    df.reset_index(inplace=True)
    df['date'] = df['Date.local'].dt.strftime('%Y-%m-%d')
    df['value'] = df['value'].astype(float, errors = 'raise')

    # Calculate mean, max and min value for each date
    Result_mean = df.groupby(['date', 'parameter'],as_index=False)['value'].mean()
    Result_max = df.groupby(['date', 'parameter'],as_index=False)['value'].max()
    Result_min = df.groupby(['date', 'parameter'],as_index=False)['value'].min()

    # Pivot the tables to wide format
    ResultWide_mean = Result_mean.pivot_table(index='date',columns='parameter', values='value')
    ResultWide_max = Result_max.pivot_table(index='date',columns='parameter', values='value')
    ResultWide_min = Result_min.pivot_table(index='date',columns='parameter', values='value')

    # Rename the columns to distinguish
    ResultWide_mean.rename(columns={"pm10": 'pm10_mean', 'pm25': 'pm25_mean'}, inplace=True)
    ResultWide_max.rename(columns={"pm10": 'pm10_max', 'pm25': 'pm25_max'}, inplace=True)
    ResultWide_min.rename(columns={"pm10": 'pm10_min', 'pm25': 'pm25_min'}, inplace=True)

    # Join mean and max first
    df_first_join = pd.merge(ResultWide_mean, ResultWide_max, left_index=True, right_index=True)

    # Join now to min
    ResultWide = pd.merge(df_first_join, ResultWide_min, left_index=True, right_index=True)

    # Change the index
    ResultWide.index.name = 'date'
    ResultWide.reset_index(inplace=True)

    return ResultWide 
    
# # Call the function
city = 'Reykjavík'
date_from = dt.strptime('01012020', "%d%m%Y").date()
date_to = dt.strptime('01122021', "%d%m%Y").date()
weather_data = fetch_weather_data(city)
#pollution_data = fetch_pollution_data(city, date_from, date_to)


In [ ]:
## INSERT CODE TO FIND CITIES

In [ ]:
## INSERT CODE TO READ CSV FILE

# Pollution comparing socio-economic measures

In order to produce a comprehensive report that aims to identify key factors that influence daily pollution in a certain city it is imperative to look into socio-economic factors. Intuativley population demographics would appear to play a big part in the pollution of certain cities. Alongside this the relative wealth of countries, their industry make-up and also education levels can all have implicit or explicit effects on pollution levels.

In order to drill down into set areas a few factors were analysed. These were chosen as we belive they are primary risk factors to the pollution levels in a city. 



In [ ]:
## INSERT CODE TO PRODUCE GRAPHS

# Population 

The population of a city has big implications as to the levels of pollution. As a city grows in size so does the density. A higher population can cause strain on a city's infrastructure requirements, including education system and waste and seweage systems. According to a study by numbeo $^{1}$ out of the worlds 20 most polluted cities, 9 were in the top 20 for population density. 

Our hypothesis is that the the effects are of population are threefold - as a city increases in size education levels lag behind,. Lower eductaion levels can mean less awareness about pollution and health. Moreover, bigger cities are usually harder to police and more red tape and legistlation to pass. Lastly, a greater population will mean higher energy usage and a greater need for energy generation, which, is usally in the form of fossil fuels. 

INSERT GRAPHS



# GDP

The GDP of a country would also have a big impact on pollution levels. Again, we have chosen to examine this measure due to the knock on or secondary effects GDP has on a pollution levels. During COP 21, it was heavility reported of China and India's desire to continue pollution given they are emerging nations. Emerging nations can sometimes view climate change and pollution level as an issue for the 'west' given high amounts of pollution during the industrial revolution in countries like the US and the UK.

Our hypthesis is that the effects of GDP are two fold. A lower level of GDP will mean lower government budgets to invest in renewable energy and clean industry, dampening the effects of pollution. In addition to this, a lower level of GDP will mean a higher proportion of economic activity related to farming and non-service sector activities. These in-turn lead to slash and burn farming, mining and excavation and also a larger mass manufacturing sector. 

INSERT GRAPHS



# Other

TELL ME WHAT OTHER MEANS AND I WILL TYPE SOMETHING UP

# Events

Lastly, events have a big impact on the pollution of a city, country and area. As touched upon during the introduction these events could be man-made or natural. Events such as the wildfire and volcanos will often produce large amounts of sulfur dioxide, carbon dioxide, and hydrogen fluoride. As we may examine, countries such as Iceland or Greece and more suseptible than others to events like this. 

Events can also be man made. We can classify COVID as an event and it was wideley reported that air pollution reduced by multiple factors during 2020 $^{2}$. This is just one example of an event causing air pollution to reduce, however, events such as strikes, legistlation changes would also have this effect. 

Contrary to this, our hypothesis is that events are mainly detremental to air quality. Sporting events being just one example that leads to a large number of people congregating in a set area leading to higher trasport pollution.

INSERT GRAPHS




$^{1}$ https://www.numbeo.com/pollution/rankings.jsp

$^{2}$ Marco Travaglio, Yizhou Yu, Rebeka Popovic, Liza Selley, Nuno Santos Leal, Luis Miguel Martins,
Links between air pollution and COVID-19 in England,

# Pollution comparing weather

Weather is also another factor that affects air pollution. As mentioned in the previous section, weather in the form of natural disaster can have a major impact on air pollution. However, the impact of weather of air pollution goes further than this.


In [ ]:
## INSERT WEATHER API CODE

# Pollution and wind

Weather systems such as wind and sunlight can have an impact on air pollution. During the 1950's London was named 'pea soup'. This was a combination of multiple multiple particulates and sulphur dioxide and occured when winds were low. Moreover, events such as acid rain are more common with greater levels of humidity and sunlight or UV rays can neautralise some pullutants through the form of a chemical reaction.

INSERT GRAPHS 


# Temperature and wind

The temperate of a country or city may also have a big impact on pollution. Hotter cities are often more prone to high levels of pollution as high temperatures are often associated with differing levels of pollution due to the movement of air. We hypothsise that hotter countries are prone to higher levels of pollution.

INSERT GRAPHS


# Regression

We aim to build a regression model to indicate the level of pollution on a set day based on multiple criteria. This will take the form of an air quality index for governements to use.




In [ ]:
## INSERT REGRESSION CODE